In [8]:
# necessary packages
import pandas as pd
from pandas.io.json import json_normalize
import gzip
import json
import multiprocessing as mp
import time
import copy
import pickle
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

## Creating a DataFrame from the Dataset

In [52]:
def convert_json(json_str):
#     converts json string to df
    return json_normalize(json.loads(json_str))

def convert_data(prefix, file_num):
    for i in range(file_num):
        suffix = str(i).zfill(12)
        filename = prefix + suffix
        
        with gzip.open(filename, "rt", encoding = "utf-8") as file:
            with mp.Pool() as pool:
                results = pool.map(convert_json, [line for line in file if line])
                mode = 'a' if i else 'w'
                pd.concat(results, sort=False).to_csv('data.csv', header = not i, mode = mode)        

In [ ]:
# convert gunzipped json files from google bigquery to a single csv
# convert_data('data/2019_01_04_', 4)

In [2]:
def load_data(filename):
    df = pd.read_csv(filename)
    df['outputs_addresses'] = df['outputs_addresses'].apply(
        lambda addr: addr[2:-2])
    df['inputs_addresses'] = df['inputs_addresses'].apply(
        lambda addr: addr[2:-2])
    return df
df = load_data('data.csv')

## Creating Address Clusters

In [1]:
def construct_clusters(df):
    clusters = []

    def cluster(group):
        addrs = group['inputs_addresses']
        new_clstr = True
        for idx, cluster in enumerate(clusters):
            for addr in addrs:
                if addr in cluster:
                    clusters[idx].update(addrs)
                    new_clstr = False
                    break
            if not new_clstr:
                break
        if new_clstr:
            clusters.append(set(addrs))

    start = time.time()
    df.groupby('tx_hash').progress_apply(cluster)
    end = time.time()
    print(f'This operation took {round((end-start) / 3600, 2)} hr.')
    return clusters
# clusters = construct_clusters(df)

In [13]:
def picklify(obj, filename):
# save obj in a pickle file for later
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
# picklify(clusters, 'clusters.pickle')

def unpicklify(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
clusters = unpicklify('clusters.pickle')

In [19]:
def clusters_are_consolidated(clusters):
    for idx, cluster in enumerate(clusters):
        for other in clusters[idx + 1:]:
            inter = cluster.intersection(other)
            if inter:
                return False
    return True

# def consolidate_clusters(clusters):
#     consolidated = clusters_are_consolidated(clusters)
    
#     while()